In [10]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [11]:
dataset = pd.read_json('cve-2016.json')

In [16]:
dataset.columns.values

array(['CVE_Items', 'CVE_data_format', 'CVE_data_numberOfCVEs',
       'CVE_data_timestamp', 'CVE_data_type', 'CVE_data_version'],
      dtype=object)

In [24]:
rev = []
j=0
for i in range(dataset.shape[0]):
    new=dataset.CVE_Items[i]
    if('baseMetricV2' in new['impact'].keys()):
        var=new['impact']['baseMetricV2']['severity']
        rev.append(var)
    else:
        rev.append('Not Exists')
    j=j+1  

In [25]:
rev=np.array(rev)
rev.shape

(9417,)

In [26]:
description = []
severity = []
scores = []

for i in range(dataset.shape[0]):
    new=dataset.CVE_Items[i]
    if('baseMetricV2' in new['impact'].keys()):
        severity.append(new['impact']['baseMetricV2']['severity'])
        scores.append(new['impact']['baseMetricV2']['cvssV2']['baseScore'])
        description.append(new['cve']['description']['description_data'][0]['value'])

In [27]:
description = np.array(description)
severity = np.array(severity)
scores = np.array(scores)

In [28]:
from bs4 import BeautifulSoup  
import re
from nltk.corpus import stopwords
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words )) 

In [29]:
# Get the number of reviews based on the dataframe column size

# Initialize an empty list to hold the clean reviews
clean_description = []

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 
for i in range(description.shape[0]):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    clean_description.append( review_to_words( description[i] ) )

In [35]:
clean_description_array = np.array(clean_description)

In [37]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1) 
train_data_features1 = vectorizer.fit_transform(clean_description_array)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features1 = train_data_features1.toarray()

In [38]:
vocab_bi = vectorizer.get_feature_names()

In [39]:
#print (vocab_bi)

In [40]:
train_data_features1.shape

(8205, 5000)

In [41]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer(smooth_idf=False)
train_data_features1 = tfidf_transformer.fit_transform(train_data_features1)
train_data_features1

<8205x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 179826 stored elements in Compressed Sparse Row format>

In [42]:
score_result = np.array(scores).astype(np.float)
score_result.shape

(8205,)

In [43]:
from sklearn import ensemble
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.cross_validation import StratifiedKFold

n_folds = 5
avg_mad = 0
skf = StratifiedKFold(score_result, n_folds)



for train_index, test_index in skf:
    X_train, X_test = train_data_features1[train_index], train_data_features1[test_index]
    y_train, y_test = score_result[train_index], score_result[test_index]
    params = {'n_estimators': 500, 'max_depth': 8, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
    clf = ensemble.GradientBoostingRegressor(**params)
    clf.fit(X_train, y_train)
    mse = mean_squared_error(y_test, clf.predict(X_test))
    mad = mean_absolute_error(y_test, clf.predict(X_test))
    #print("MSE: %.4f" % mse)
    avg_mad += mad
    print("MAE: %.4f" % mad)
print("avg",avg_mad/n_folds)  

MAE: 0.9860
MAE: 0.9221
MAE: 0.9988
MAE: 0.9547
MAE: 1.0171
avg 0.9757471145780707


In [19]:
temp = description_features[30:31,0:500]

NameError: name 'description_features' is not defined

In [20]:
clf.predict(temp)

NameError: name 'clf' is not defined

In [21]:
scores[30]

5.0